# DicoCo

Maxime Delerue

In [52]:
# import necessaire à la création des widgets
from IPython.display import display, clear_output, HTML, Markdown, Audio
import ipywidgets as widgets
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import pandas as pd
import os
import httplib2
import requests
from requests_html import HTMLSession
from bs4 import BeautifulSoup, SoupStrainer

session = HTMLSession()

In [3]:
buchanan = pd.read_csv("./../data/dataframe/BuchananRich.csv", sep=",", encoding="utf-8")
buchanan = buchanan.drop(columns =['hwd'], axis=1)
buchanan

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,IPA,MotVedette,pronSampa,ipa,sizePron
0,3,aʹbăcus,NaN,NaN,NaN,n.,2.0,19,NaN,abacus,{ b @ k @ s,æbəkəs,7
1,3,ābăʹndon,F.,NaN,NaN,v.,5.0,19,NaN,abandon,@ b { n d @ n,əbændən,8
2,4,ābăʹndŏned,NaN,NaN,NaN,a.,6.0,19,NaN,abandoned,@ b { n d @ n d,əbændənd,10
3,2,ăbăʹſh,F.,NaN,NaN,v.,9.0,19,NaN,abash,@ b { S,əbæʃ,6
4,2,ăbāʹte,F.,NaN,NaN,v.,10.0,19,NaN,abate,@ b eI t,əb?t,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10328,2,zĕʹnith,NaN,NaN,NaN,n.,11.0,241,NaN,zenith,z i: n @ T,ziːnəθ,7
10329,1,zĕſt,NaN,NaN,NaN,n.,13.0,241,NaN,zest,z E s t,zɛst,4
10330,1,zĭnk,NaN,NaN,NaN,n.,14.0,241,NaN,zink,z I N k,zɪŋk,4
10331,3,zōʹdĭac,NaN,NaN,NaN,n.,16.0,241,NaN,zodiac,z @U 4 i: { k,zoʊɾiːæk,7


In [50]:
# Fonction pour récupérer l'URL du mp3 réalisé dans la partie 1 

def recupMP3(mot):
    response = session.get("https://www.ldoceonline.com/dictionary/" + mot)
    html_doc = response.content
    soup = BeautifulSoup(html_doc,'html.parser')
    souplettesBody = soup.find('body')
    findedUK = souplettesBody.findAll('span',{"class":"speaker brefile fas fa-volume-up hideOnAmp"})
    findedUS = souplettesBody.findAll('span',{"class":"speaker amefile fas fa-volume-up hideOnAmp"})
    for v in findedUK:
        mp3ENG = v.get_attribute_list('data-src-mp3')
        break
    for v in findedUS:
        mp3US = v.get_attribute_list('data-src-mp3')
        break
    characters = "'[]"
    string = mp3ENG[0]
    for x in range(len(characters)):
        stringENG = string.replace(characters[x],"")
    string = mp3US[0]
    for x in range(len(characters)):
        stringUS = string.replace(characters[x],"")
    return stringENG,stringUS

In [10]:
# definition du widget permettant de taper un texte 
textTarget = widgets.Text(
       value="",
       description='filtre hwd',
       layout={'width': '300px'} )

ipaTarget = widgets.Text(
       value="",
       description='filtre IPA',
       layout={'width': '300px'} )

nbColonne = widgets.IntSlider(
    max = 10, value = 1,
    description='Limit Data', 
    disabled=False
)

In [18]:
# définition du widget button permettant de lancer l'extraction
button = widgets.Button(description='Chercher')

buchananFiltered = buchanan[buchanan['MotVedette'].str.contains(textTarget.value, na=False, regex=True)]
buchananFiltered = buchananFiltered[buchananFiltered['ipa'].str.contains(ipaTarget.value, na=False, regex=True)]

# définition du widget permettant l'affichage d'une sortie
cambridge = widgets.Output()
longman = widgets.Output()
linguee = widgets.Output()
tableau = widgets.Output()
cam_audio_uk = widgets.Output()
cam_audio_us = widgets.Output()

def removeRegex(inputString):
    outputstring = ""
    for c in inputString:
        if c.isalpha():
            outputstring = outputstring + c
    return outputstring

# spécifier les actions à réaliser lors de l'appui sur le bouton
def on_button_clicked(_):
    
    
    #tableau filtré
    if (textTarget.value != ''): 
        buchananFiltered = buchanan[buchanan['MotVedette'].str.contains(textTarget.value, na=False, regex=True)]
        if (ipaTarget.value != ''):
            buchananFiltered = buchananFiltered[buchananFiltered['ipa'].str.contains(ipaTarget.value, na=False, regex=True)]
    elif (ipaTarget.value != ''):
        buchananFiltered = buchanan[buchanan['ipa'].str.contains(ipaTarget.value, na=False, regex=True)]
    buchananFiltered = buchananFiltered[0:nbColonne.value]
    
    # récupérer le texte saisi par l'utilisateur
    targetWord = removeRegex(textTarget.value)
    
    linkAudio = recupMP3(targetWord)
                
    with longman:
        clear_output()        
        urlLongman = "https://www.ldoceonline.com/dictionary/"
        urlTarget = urlLongman+targetWord
        display(HTML('<a href="'+urlTarget+'" target="_blank" >'+'longman : '+targetWord+'</a>'))

    with linguee:
        clear_output()
        urlLinguee = "https://www.linguee.fr/anglais-francais/traduction/"
        urlTarget = urlLinguee+targetWord+'.html'
        display(HTML('<a href="'+urlTarget+'" target="_blank" >'+'linguee : '+targetWord+'</a>'))
        
    with cam_audio_uk:
        clear_output()        
        display(HTML("<audio controls src='str(linkAudio[0])'></audio>"))
    
    with cam_audio_us:
        clear_output()
        display(HTML("<audio controls src='str(linkAudio[1])'></audio>"))

    with tableau:
        clear_output()
        display(buchananFiltered)
        
button.on_click(on_button_clicked)

In [48]:
#Bouton pour scrapper
button2 = widgets.Button(description='Collecter')

def on_button2_clicked(_):
    targetWord = removeRegex(textTarget.value)
    linkAudio = recupMP3(targetWord)
    session = HTMLSession()
    urlUK = linkAudio[0]
    urlUS = linkAudio[1]
    responseUK = session.get(urlUK)
    responseUS = session.get(urlUS)
    with open("./../data/audio/"+ targetWord +'UK.mp3', 'wb') as f:
        f.write(responseUK.content)
    with open("./../data/audio/"+ targetWord +'US.mp3', 'wb') as f:
        f.write(responseUS.content)
        
button2.on_click(on_button2_clicked)

In [23]:
# définition du widget permettant l'affichage d'une sortie
out1 = widgets.Output()
out2 = widgets.Output()

In [51]:
# les widgets de filtre input
vbFiltres = widgets.HBox([textTarget,ipaTarget,nbColonne, button])

# les widgets d'output
vbOuts = widgets.VBox([tableau,longman,linguee])

#boucle à corriger
for m in buchananFiltered['MotVedette'][0:3]:
    labelUK = widgets.Label(value="Britain : " + textTarget.value)
    labelUS = widgets.Label(value="American : " + textTarget.value)
    vAudios = widgets.HBox([widgets.VBox([labelUK,cam_audio_uk]), widgets.VBox([labelUS, cam_audio_us]),button2])

hbAllWidgets = widgets.VBox([vbFiltres,vbOuts, vAudios])
hbAllWidgets